In [2]:
# Model comparison
from pathlib import Path
import sys

sys.path.append(str(Path(sys.path[0]).parent))

In [4]:
from isegm.model.is_hrnet_model import HRNetModel


def params_hrnet_18s(**kwargs):
    model = HRNetModel(width=18, ocr_width=48, small=True, with_aux_output=True,
                       use_disks=True, norm_radius=5, with_prev_mask=True)
    return model


def params_hrnet_18(**kwargs):
    model = HRNetModel(width=18, ocr_width=64, with_aux_output=True,
                       use_disks=True, norm_radius=5, with_prev_mask=True)
    return model


def params_hrnet_32(**kwargs):
    model = HRNetModel(width=32, ocr_width=128, with_aux_output=True, use_disks=True, 
                       norm_radius=5, with_prev_mask=True)
    return model


def get_params_count(model):
    return sum(param.numel() for param in model.parameters())

hrnet18s = params_hrnet_18s()
hrnet18 = params_hrnet_18()
hrnet32 = params_hrnet_32()

for model in [hrnet18s, hrnet18, hrnet32]:
    print('-------------')
    print('{:.3f} M'.format(get_params_count(model) / 1000 / 1000))

-------------
4.211 M
-------------
10.023 M
-------------
30.942 M


In [5]:
# Compute flops
import torch
from thop import profile

input = torch.randn(1, 4, 448, 448)
point = torch.randn(1, 2, 3)

output = hrnet18s(input, point)


/playpen-raid/qinliu/tools/anaconda3/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


debug:  torch.Size([1, 64, 224, 224]) torch.Size([1, 3, 448, 448])


RuntimeError: The size of tensor a (224) must match the size of tensor b (448) at non-singleton dimension 3